In [1]:
import sys,os
os.chdir('../../')
sys.path.append('./src/data')
sys.path.append('./src/events')
sys.path.append('./src/probability')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from PINGU.importer import *
from PINGU.processer import *
from PINGU.main import get_all_events,get_events
from functions import perform_chisq, dc_params_nsi, dc_params
from PINGU.event_processing import list_of_params_nsi, return_precomputed_nsi, get_deltachi, normalize_events
from scipy.stats import chi2
import pickle
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 100
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')
np.set_printoptions(linewidth=200)

In [2]:
ndim = 3
pid = 1
precomputed_events = True
dc_params_nsi['dm_41'] = 0.93
dc_params['dm_41'] = 0.93
emm_range = np.linspace(0,0,1)
s24_range = np.linspace(0,0,1)
emt_range = np.linspace(-1e-2,1e-2,10)
param_list = list_of_params_nsi(dc_params_nsi,s24_range, emm_range,emt_range)

#for p in param_list: # Assert all dicts returned from param_list have precomputed probs.
#    assert is_precomputed_nsi(N=N,ndim=ndim, dict=p,check=False)



param_list = return_precomputed_nsi(pid,ndim,params=param_list)
emm_range = np.sort(np.unique(np.array([p['e_mm'] for p in param_list])))
s24_range = np.sin(2*np.sort(np.unique(np.array([p['theta_24'] for p in param_list]))))**2
emt_range = np.sort(np.unique(np.array([p['e_mt'] for p in param_list])))

print(emm_range)
print(s24_range)
print(emt_range)

#print(param_list[1])

[0.]
[0.]
[-0.01       -0.00777778 -0.00555556 -0.00333333 -0.00111111  0.00111111  0.00333333  0.00555556  0.00777778  0.01      ]


In [3]:
if not precomputed_events:
    print('Computing events')
    from multiprocessing import Pool
    p = Pool()
    data = [(p, pid,False) for p in param_list]
    H1_events_list = p.starmap(get_all_events, data)
    p.close()
    pickle.dump(H1_events_list,open(f'./pre_computed/H1_PINGU_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','wb'))

    H0_events = get_all_events(params=param_list[0], pid=pid, null=True)
    pickle.dump(H0_events,open(f'./pre_computed/H0_PINGU_{pid}.p','wb'))

H1_events_list = pickle.load(open(f'./pre_computed/H1_gen2_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','rb'))
H0_events = pickle.load(open(f'./pre_computed/H0_PINGU_{pid}.p','rb'))


H0_normalized, H1_list_normalized = normalize_events(H0_events,H1_events_list,pid)

In [6]:
sigma_a = 0.25
sigma_b = 0.04
f = 0
deltachi, best_index, H1_chisq = get_deltachi(H1_list_normalized, H0_normalized,emt_range,emm_range,0,[sigma_a,sigma_b],f,x0=[1])
#s24_cl90, s24_cl99, emm_cl90, emm_cl99 = get_contour(deltachi.T, emt_range,emm_range,2)
#best_fit_coords = (param_list[best_index]['e_mt'], param_list[best_index]['e_mm'])

In [10]:
deltachi

array([[ 0.        ],
       [12.98547712],
       [25.3298408 ],
       [36.40792578],
       [46.62030861],
       [55.2711052 ],
       [62.28192845],
       [68.1764869 ],
       [71.94664659],
       [74.16301697]])

In [20]:
from gen2.event_processing import perform_chisq
np.array([perform_chisq(H1_norm, H0_events,sigma_syst=0,z=0, sigma_a=0.25,sigma_b=0.04,sigma_g=0, x0=[1])[0] for H1_norm in H1_events_list[0:3]])

array([0.98533481, 0.98533481, 0.98533481])